In [1]:
from transformers import T5Tokenizer
from gluonnlp.data.tokenizers import SentencepieceTokenizer
from gluonnlp.data.vocab import Vocab

In [2]:
trans = T5Tokenizer.from_pretrained("t5-base")

In [3]:
# extra_ids = 100
# special_tokens = {'extra_{}_token'.format(i): '<extra_id_{}>'.format(i) for i in range(extra_ids)}
# special_tokens['eos_token'] = '</s>'
# special_tokens['unk_token'] = '<unk>'
# special_tokens['pad_token'] = '<pad>'
# gluon = SentencepieceTokenizer(
#     model_path=trans.vocab_file,
#     vocab=Vocab(trans.get_vocab(), **special_tokens)
# )

In [17]:
import tempfile
import os

from gluonnlp.base import get_repo_url
from gluonnlp.data import load_vocab
from gluonnlp.utils.misc import download


with tempfile.TemporaryDirectory() as dir_path: 
    model_path = os.path.join(dir_path, 't5_spm.model')
    download(
        url=get_repo_url() + 'tokenizer_test_models/sentencepiece/case_t5/test_t5spm-5f05e7.model',
        path=model_path
    )
    vocab_path = os.path.join(dir_path, 't5_spm_vocab.json')
    download(
        url=get_repo_url() + 'tokenizer_test_models/sentencepiece/case_t5/test_t5spm_vocab-a9d819.json', 
        path=vocab_path
    )
    gluon = SentencepieceTokenizer(
        model_path=model_path, 
        vocab=load_vocab(vocab_path)
    )
    print(gluon._vocab.special_tokens)
    print(gluon._vocab['<extra_id_0>'], gluon._vocab['<extra_id_99>'])
    for token in gluon._vocab.special_tokens:
        piece_id = gluon._sp_model.piece_to_id(token)
        print(token, piece_id)
        if gluon._sp_model.is_unknown(piece_id):
            assert gluon._vocab[token] == gluon._sp_model.unk_id()
    os.remove(model_path)
    os.remove(vocab_path)

100%|██████████| 792k/792k [00:00<00:00, 11.7MiB/s]
100%|██████████| 476k/476k [00:00<00:00, 8.02MiB/s]Downloading /tmp/tmpptcvki94/t5_spm.model from s3://gluonnlp-numpy-data/tokenizer_test_models/sentencepiece/case_t5/test_t5spm-5f05e7.model...

['<pad>', '</s>', '<unk>', '<extra_id_99>', '<extra_id_98>', '<extra_id_97>', '<extra_id_96>', '<extra_id_95>', '<extra_id_94>', '<extra_id_93>', '<extra_id_92>', '<extra_id_91>', '<extra_id_90>', '<extra_id_89>', '<extra_id_88>', '<extra_id_87>', '<extra_id_86>', '<extra_id_85>', '<extra_id_84>', '<extra_id_83>', '<extra_id_82>', '<extra_id_81>', '<extra_id_80>', '<extra_id_79>', '<extra_id_78>', '<extra_id_77>', '<extra_id_76>', '<extra_id_75>', '<extra_id_74>', '<extra_id_73>', '<extra_id_72>', '<extra_id_71>', '<extra_id_70>', '<extra_id_69>', '<extra_id_68>', '<extra_id_67>', '<extra_id_66>', '<extra_id_65>', '<extra_id_64>', '<extra_id_63>', '<extra_id_62>', '<extra_id_61>', '<extra_id_60>', '<extra_id_59>', '<extra_id_58>', '<extra_id_5

AssertionError: 

In [30]:
SUBWORD_TEST_SAMPLES = ["Hello, y'all! How are you Ⅷ 😁 😁 😁 ?",
                        'GluonNLP is great！！！!!!',
                        "GluonNLP-Amazon-Haibin-Leonard-Sheng-Shuai-Xingjian...../:!@# 'abc'"]

print(gluon.encode(SUBWORD_TEST_SAMPLES, int))
print(trans(SUBWORD_TEST_SAMPLES))

print(gluon.encode_with_offsets(SUBWORD_TEST_SAMPLES))

ints = gluon.encode(SUBWORD_TEST_SAMPLES, int)
gluon.decode(ints)

([['▁Hello', ',', '▁', 'y', "'", 'all', '!', '▁How', '▁are', '▁you', '▁VIII', '▁', '😁', '▁', '😁', '▁', '😁', '▁', '?'], ['▁', 'Glu', 'on', 'N', 'LP', '▁is', '▁great', '!', '!!!!!'], ['▁', 'Glu', 'on', 'N', 'LP', '-', 'Am', 'a', 'zon', '-', 'H', 'a', 'i', 'bin', '-', 'Le', 'on', 'ard', '-', 'She', 'ng', '-', 'Sh', 'u', 'a', 'i', '-', 'X', 'ing', 'j', 'i', 'an', '.....', '/', ':', '!', '@', '#', '▁', "'", 'a', 'b', 'c', "'"]], [[(0, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 12), (12, 13), (13, 17), (17, 21), (21, 25), (25, 27), (27, 28), (28, 29), (29, 30), (30, 31), (31, 32), (32, 33), (33, 34), (34, 35)], [(0, 0), (0, 3), (3, 5), (5, 6), (6, 8), (8, 11), (11, 17), (17, 18), (18, 23)], [(0, 0), (0, 3), (3, 5), (5, 6), (6, 8), (8, 9), (9, 11), (11, 12), (12, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 22), (22, 23), (23, 25), (25, 27), (27, 30), (30, 31), (31, 34), (34, 36), (36, 37), (37, 39), (39, 40), (40, 41), (41, 42), (42, 43), (43, 44), (44, 47), (47, 48), (48, 49), (49, 51), (

In [35]:
from gluonnlp.base import get_repo_url
from gluonnlp.data import Vocab, load_vocab
from gluonnlp.utils.misc import download
import os

In [36]:
vocab_path = os.path.join('/home/ubuntu/yongyiw/temp', 't5_spm.json')
download(
    url=get_repo_url() + 'tokenizer_test_models/sentencepiece/case_t5/test_t5spm-821614.json', 
    path=vocab_path
)

100%|██████████| 540k/540k [00:00<00:00, 7.96MiB/s]

'/home/ubuntu/yongyiw/temp/t5_spm.json'

In [37]:
v = load_vocab(vocab_path)

AttributeError: 'str' object has no attribute 'get'

In [6]:
import json

In [7]:
vocab = gluon._vocab.to_json()
vocab = json.loads(vocab)
with open('/home/ubuntu/yongyiw/temp/test_t5spm.json', 'w') as vocab_file: 
    json.dump(vocab, vocab_file)

In [30]:
def noise_span_to_unique_sentinel(tokens, noise_mask, vocab):
  """Replace each run of consecutive noise tokens with a different sentinel.
  The idea here is to be able to align the dropped spans in the inputs
  with the markers in the targets.
  We want to generate training examples like
  "We hold X to be Y that" -> "X these truths Y self evident Z"
  Sentinels assigned in decreasing order within the sequence starting at
  vocabulary.size - 1.  That is, we appropriate the last tokens in the
  vocabulary for additional use as sentinels.
  TODO(noam): we may want to try enlarging the vocabulary and leaving room
  for the sentinels instead.  However, this requires enlarging the embedding
  tables in the model, so that is a bigger change.
  Args:
    tokens: a 1d integer Tensor
    noise_mask: a boolean Tensor with the same shape as tokens
    vocabulary: a vocabulary.Vocabulary
    seeds: an unused int32 Tensor
  Returns:
    a Tensor with the same shape and dtype as tokens
  """
  vocab_size = vocab
  prev_token_is_noise = tf.pad(noise_mask[:-1], [[1, 0]])

  first_noise_tokens = tf.logical_and(
      noise_mask, tf.logical_not(prev_token_is_noise))
  subsequent_noise_tokens = tf.logical_and(noise_mask, prev_token_is_noise)

  sentinel = vocab_size - tf.cumsum(tf.cast(first_noise_tokens, tokens.dtype))

  tokens = tf.where(first_noise_tokens, sentinel, tokens)
  return tf.boolean_mask(tokens, tf.logical_not(subsequent_noise_tokens))

In [31]:
noise_span_to_unique_sentinel(
    tf.constant([1, 2, 3, 4, 5, 6, 7, 8]), 
    tf.constant([True, False, False, False, True, True, False, True]), 
    1000
)

<tf.Tensor: shape=(7,), dtype=int32, numpy=array([999,   2,   3,   4, 998,   7, 997], dtype=int32)>